In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import os
import shutil

# --- CONFIGURATION ---
DATA_DIR = "../data"
SEED = 42  # The God Constant: Ensures every student gets the exact same "random" data
NUM_USERS = 100_000
NUM_PRODUCTS = 1_000
NUM_ORDERS = 500_000
NUM_CLICKS = 2_000_000

# Initialize
fake = Faker()
Faker.seed(SEED)
np.random.seed(SEED)

def setup_directories():
    # If the directory doesn't exist, create it
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
        print(f"📂 Created new directory: {DATA_DIR}")
        return

    # If it exists (and might be a mount point), empty it carefully
    print(f"🧹 Cleaning existing directory: {DATA_DIR}...")
    for filename in os.listdir(DATA_DIR):
        file_path = os.path.join(DATA_DIR, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path) # Delete file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path) # Delete subfolder
        except Exception as e:
            print(f"⚠️ Failed to delete {file_path}. Reason: {e}")
            
    print(f"✨ Directory cleaned: {DATA_DIR}")

# --- 1. USERS (The Dimension Table) ---
def generate_users():
    print(f"👤 Generating {NUM_USERS} Users...")
    
    # We need a mix of high and low cardinality for Indexing/Compression demos
    data = {
        'user_id': np.arange(1, NUM_USERS + 1),
        'name': [fake.name() for _ in range(NUM_USERS)],
        'email': [fake.unique.email() for _ in range(NUM_USERS)],
        
        # LOW CARDINALITY: Perfect for Bitmap Index & RLE Compression demos
        # Only 3 unique values across 100k rows
        'account_status': np.random.choice(['ACTIVE', 'INACTIVE', 'PREMIUM'], NUM_USERS, p=[0.70, 0.25, 0.05]),
        
        # HIGH CARDINALITY: Good for B-Tree demos
        'signup_date': [fake.date_time_this_decade() for _ in range(NUM_USERS)],
        
        # BOOLEAN: For "Bit packing" demos
        'is_email_verified': np.random.choice([True, False], NUM_USERS)
    }
    
    df = pd.DataFrame(data)
    
    # Save as CSV (Row-based, slow to read)
    df.to_csv(f"{DATA_DIR}/users.csv", index=False)
    
    # Save as Parquet (Column-based, fast to read)
    df.to_parquet(f"{DATA_DIR}/users.parquet")
    print("   -> Saved users.csv and users.parquet")
    return df

# --- 2. PRODUCTS (The Small Table) ---
def generate_products():
    print(f"📦 Generating {NUM_PRODUCTS} Products...")
    
    data = {
        'product_id': np.arange(1, NUM_PRODUCTS + 1),
        'product_name': [fake.catch_phrase() for _ in range(NUM_PRODUCTS)],
        'category': np.random.choice(['Electronics', 'Books', 'Home', 'Garden', 'Toys'], NUM_PRODUCTS),
        'price': np.round(np.random.uniform(5.0, 500.0, NUM_PRODUCTS), 2)
    }
    
    df = pd.DataFrame(data)
    df.to_csv(f"{DATA_DIR}/products.csv", index=False)
    df.to_parquet(f"{DATA_DIR}/products.parquet")
    return df

# --- 3. ORDERS (The Fact Table - Uniform Distribution) ---
def generate_orders(users_df, products_df):
    print(f"🛒 Generating {NUM_ORDERS} Orders (Uniform Distribution)...")
    
    # Randomly assign users and products
    user_ids = users_df['user_id'].values
    product_ids = products_df['product_id'].values
    
    data = {
        'order_id': np.arange(1, NUM_ORDERS + 1),
        'user_id': np.random.choice(user_ids, NUM_ORDERS),
        'product_id': np.random.choice(product_ids, NUM_ORDERS),
        'quantity': np.random.randint(1, 5, NUM_ORDERS),
        'order_date': [fake.date_time_this_year() for _ in range(NUM_ORDERS)]
    }
    
    df = pd.DataFrame(data)
    
    # SORTED VERSION: For "Clustered Index" & "Sequential Read" demos
    # Reading this from disk will be fast because it's physically ordered by date
    df_sorted = df.sort_values(by='order_date')
    df_sorted.to_csv(f"{DATA_DIR}/orders_sorted.csv", index=False)
    
    # SHUFFLED VERSION: For "Heap Scan" & "Random Seek" demos
    # Reading this chronologically will force the disk head to jump around
    df_shuffled = df.sample(frac=1).reset_index(drop=True)
    df_shuffled.to_csv(f"{DATA_DIR}/orders_shuffled.csv", index=False)
    
    print("   -> Saved orders_sorted.csv (Sequential I/O) and orders_shuffled.csv (Random I/O)")

# --- 4. CLICKSTREAM (The Big Data - SKEWED Distribution) ---
def generate_clickstream(users_df):
    print(f"🖱️ Generating {NUM_CLICKS} Click Events (Highly Skewed)...")
    
    # THE SKEW: We want to break distributed joins (Shuffle).
    # User ID 1 (The "Bot") generates 50% of all traffic.
    # The other 50% is spread among the remaining 99,999 users.
    
    skew_count = int(NUM_CLICKS * 0.50)
    normal_count = NUM_CLICKS - skew_count
    
    heavy_hitter = np.full(skew_count, 1) # 1 Million clicks for User #1
    long_tail = np.random.choice(users_df['user_id'].values, normal_count)
    
    # Combine and shuffle so the file looks "random" but is statistically cursed
    all_users = np.concatenate([heavy_hitter, long_tail])
    np.random.shuffle(all_users)
    
    data = {
        'event_id': np.arange(1, NUM_CLICKS + 1),
        'user_id': all_users,
        'url': [fake.uri_path() for _ in range(NUM_CLICKS)],
        'timestamp': [fake.date_time_this_year() for _ in range(NUM_CLICKS)]
    }
    
    df = pd.DataFrame(data)
    
    # Partitioned Write: Useful for "Partition Pruning" demos
    # We save this as parquet
    df.to_parquet(f"{DATA_DIR}/clickstream.parquet")
    print("   -> Saved clickstream.parquet (Contains the 'Justin Bieber' skew)")

# --- EXECUTION ---
if __name__ == "__main__":
    setup_directories()
    users = generate_users()
    products = generate_products()
    generate_orders(users, products)
    generate_clickstream(users)
    print("\n✅ DATA GENERATION COMPLETE. The Universe is ready.")

🧹 Cleaning existing directory: ../data...
✨ Directory cleaned: ../data
👤 Generating 100000 Users...
   -> Saved users.csv and users.parquet
📦 Generating 1000 Products...
🛒 Generating 500000 Orders (Uniform Distribution)...
   -> Saved orders_sorted.csv (Sequential I/O) and orders_shuffled.csv (Random I/O)
🖱️ Generating 2000000 Click Events (Highly Skewed)...
   -> Saved clickstream.parquet (Contains the 'Justin Bieber' skew)

✅ DATA GENERATION COMPLETE. The Universe is ready.
